In [1]:
import cv2
import numpy as np
import kineticstoolkit.lab as ktk
from glob import glob
from IPython.display import display

In [2]:
# --- Select multiple files from dropdown ---
from glob import glob
import os


def get_all_files(folder_path, extension):
    search_path = os.path.join(folder_path, "**", f"*.{extension}")
    return sorted(glob(search_path, recursive=True))


In [4]:
folder_videos = r"C:\hackaton\new\20241021\camera"
folder_c3d = r"C:\hackaton\new\20241021\c3d"

mp4_files = get_all_files(folder_videos, "mp4")
xml_files = get_all_files(folder_videos, "xml")
c3d_files = get_all_files(folder_c3d, "c3d")


print("MP4 FILES:")
print(mp4_files)
print (xml_files)
print(c3d_files)

MP4 FILES:
['C:\\hackaton\\new\\20241021\\camera\\C0057.MP4', 'C:\\hackaton\\new\\20241021\\camera\\C0058.MP4']
['C:\\hackaton\\new\\20241021\\camera\\C0057M01.XML', 'C:\\hackaton\\new\\20241021\\camera\\C0058M01.XML']
["C:\\hackaton\\new\\20241021\\c3d\\T2G'002_20241021_Rehab_D4_07.c3d", "C:\\hackaton\\new\\20241021\\c3d\\T2G'002_20241021_Rehab_D4_static.c3d", 'C:\\hackaton\\new\\20241021\\c3d\\T2G002_20241021_Rehab_D4_01.c3d', 'C:\\hackaton\\new\\20241021\\c3d\\T2G002_20241021_Rehab_D4_06.c3d']


In [12]:
import numpy as np
from scipy.signal import correlate

def normalize_signal(signal):
    return (signal - np.mean(signal)) / np.std(signal)


def estimate_offset(video_signal, c3d_signal, fps_video):
    # Normalize
    video_signal = (video_signal - np.mean(video_signal)) / np.std(video_signal)
    c3d_signal = (c3d_signal - np.mean(c3d_signal)) / np.std(c3d_signal)

    min_len = min(len(video_signal), len(c3d_signal))
    v = video_signal[:min_len]
    c = c3d_signal[:min_len]

    correlation = correlate(v, c, mode='full')
    lags = np.arange(-min_len + 1, min_len)
    best_lag = lags[np.argmax(correlation)]
    offset_seconds = best_lag / fps_video

    return offset_seconds, best_lag, correlation, lags



In [13]:
import cv2
import numpy as np

def extract_video_motion_signal(video_path, max_frames=1000):
    cap = cv2.VideoCapture(video_path)
    motion_signal = []

    ret, prev = cap.read()
    if not ret:
        raise ValueError(f"Cannot read first frame from video: {video_path}")
    prev_gray = cv2.cvtColor(prev, cv2.COLOR_BGR2GRAY)

    while ret and len(motion_signal) < max_frames:
        ret, frame = cap.read()
        if not ret:
            break
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        diff = cv2.absdiff(prev_gray, gray)
        motion = np.sum(diff)  # Sum of pixel differences
        motion_signal.append(motion)
        prev_gray = gray

    cap.release()
    return np.array(motion_signal)


In [14]:
import ezc3d

def extract_c3d_motion_signal(c3d_path):
    c3d = ezc3d.c3d(c3d_path)
    points = c3d["data"]["points"][:3]  # X, Y, Z only (ignore residual)

    # Compute per-frame total motion across all markers
    diffs = np.diff(points, axis=2)  # shape: (3, markers, frames-1)
    motion = np.linalg.norm(diffs, axis=0)  # Euclidean norm per marker per frame
    total_motion = np.sum(motion, axis=0)  # Total motion per frame
    return total_motion


In [15]:
import matplotlib.pyplot as plt

offsets = []

for video_path in mp4_files:
    for c3d_path in c3d_files:
        video_name = os.path.basename(video_path)
        c3d_name = os.path.basename(c3d_path)
        print(f"Comparing: {video_name} vs {c3d_name}")

        try:
            video_signal = extract_video_motion_signal(video_path)
            c3d_signal = extract_c3d_motion_signal(c3d_path)

            offset_sec, lag, correlation, lags = estimate_offset(video_signal, c3d_signal, fps_video=15)

            # Save result
            offsets.append({
                "video": video_name,
                "c3d": c3d_name,
                "offset_seconds": offset_sec,
                "lag_frames": lag
            })

            # Plot and save convolution graph
            plt.figure(figsize=(10, 4))
            plt.plot(lags, correlation)
            plt.title(f"Cross-Correlation: {video_name} vs {c3d_name}\nOffset = {offset_sec:.2f}s")
            plt.xlabel("Lag (frames)")
            plt.ylabel("Correlation")
            plt.axvline(lag, color="red", linestyle="--", label=f"Best Lag: {lag}")
            plt.legend()
            plt.tight_layout()

            # Save plot (optional)
            plot_name = f"corr_{video_name.replace('.mp4','')}_{c3d_name.replace('.c3d','')}.png"
            plt.savefig(plot_name)
            plt.close()

        except Exception as e:
            print(f"❌ Failed: {video_name} vs {c3d_name} — {e}")



Comparing: C0057.MP4 vs T2G'002_20241021_Rehab_D4_07.c3d
Comparing: C0057.MP4 vs T2G'002_20241021_Rehab_D4_static.c3d
Comparing: C0057.MP4 vs T2G002_20241021_Rehab_D4_01.c3d
Comparing: C0057.MP4 vs T2G002_20241021_Rehab_D4_06.c3d
Comparing: C0058.MP4 vs T2G'002_20241021_Rehab_D4_07.c3d
Comparing: C0058.MP4 vs T2G'002_20241021_Rehab_D4_static.c3d
Comparing: C0058.MP4 vs T2G002_20241021_Rehab_D4_01.c3d
Comparing: C0058.MP4 vs T2G002_20241021_Rehab_D4_06.c3d


In [16]:
import pandas as pd

df_offsets = pd.DataFrame(offsets)
print(df_offsets)


       video                                   c3d  offset_seconds  lag_frames
0  C0057.MP4      T2G'002_20241021_Rehab_D4_07.c3d      -66.600000        -999
1  C0057.MP4  T2G'002_20241021_Rehab_D4_static.c3d       -7.466667        -112
2  C0057.MP4       T2G002_20241021_Rehab_D4_01.c3d      -18.600000        -279
3  C0057.MP4       T2G002_20241021_Rehab_D4_06.c3d      -66.600000        -999
4  C0058.MP4      T2G'002_20241021_Rehab_D4_07.c3d      -66.600000        -999
5  C0058.MP4  T2G'002_20241021_Rehab_D4_static.c3d      -15.200000        -228
6  C0058.MP4       T2G002_20241021_Rehab_D4_01.c3d      -29.933333        -449
7  C0058.MP4       T2G002_20241021_Rehab_D4_06.c3d      -66.600000        -999
